In [1]:
import cv2
import matplotlib.pyplot as plt
from os.path import isfile
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
import os
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Dataset
from torchvision import transforms
from torch.optim import Adam, SGD, RMSprop
import time
from torch.autograd import Variable
import torch.functional as F
from tqdm import tqdm
from sklearn import metrics
import urllib
import pickle
import cv2
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torchvision import models
import seaborn as sns
import random
import sys
import gc

LR = 1e-4
BATCH_SIZE = 32
EPOCHS = 20
device = 'cuda'

In [2]:
from dataset import RSNADataset, HAM10000Dataset,AptosDataset
from utils.utils import EarlyStopper, get_dataset


train_loader, val_loader, test_loader = get_dataset('APTOS','./dataset/224_data/DR/aptos/""./dataset/224_data/DR/aptos/aptos_dataset_splits.csv', "Minimal", "ImageNet", 224, BATCH_SIZE, 12)


['./dataset/224_data/DR/aptos/', './dataset/224_data/DR/aptos/aptos_dataset_splits.csv']
APTOS 2929 366 367


In [3]:
model = models.resnet50(pretrained = True)
model.fc = nn.Linear(in_features=2048, out_features=1, bias=True)
# model.avgpool = GeM()
model.to('cuda')

criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr = LR, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 50, T_mult=1, eta_min=0, last_epoch=- 1, verbose=False)

In [4]:
best = 1e-10
for i in range(1, EPOCHS + 1):
    train_loss = []
    val_loss = []

    model.train()
    for idx, (X_batch, y_batch) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        # print(y_batch)
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        output = model(X_batch)
        loss = criterion(output.float(), y_batch.float())
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()
    model.eval()

    for idx, (X_batch, y_batch) in enumerate(tqdm(val_loader)):
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        output = model(X_batch)
        loss = criterion(output.float(), y_batch.float())
        val_loss.append(loss.item())
    avg_loss = np.mean(val_loss)
    if avg_loss < best:
        best = val_loss
        torch.save(model.state_dict(), 'model.bin')
    print(f'Epoch : {i}')
    print(f'Training loss : {np.mean(train_loss)}')
    print(f'Valid loss : {avg_loss}')

100%|██████████| 12/12 [00:01<00:00,  6.16it/s]


Epoch : 1
Training loss : 0.23667213255944458
Valid loss : 0.13165513860682646


100%|██████████| 12/12 [00:02<00:00,  5.98it/s]


Epoch : 2
Training loss : 0.14806267882332855
Valid loss : 0.12503096585472426


100%|██████████| 12/12 [00:01<00:00,  6.19it/s]


...

Epoch : 18
Training loss : 0.01879420570280079
Valid loss : 0.13580494249860445


100%|██████████| 12/12 [00:01<00:00,  6.05it/s]


Epoch : 19
Training loss : 0.020374703074238547
Valid loss : 0.13013203193744025


100%|██████████| 12/12 [00:01<00:00,  6.23it/s]

Epoch : 20
Training loss : 0.021195768540644127
Valid loss : 0.12226242199540138


In [5]:
model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for (X, y) in tqdm(val_loader):
        X = X.to(device)
        output = model(X)
        y_pred.extend(output)
        y_true.extend(y)

100%|██████████| 12/12 [00:01<00:00,  6.00it/s]


In [6]:
y_pred

[tensor([2.2935], device='cuda:0'),
 tensor([2.1443], device='cuda:0'),
 tensor([0.0589], device='cuda:0'),
 tensor([1.1116], device='cuda:0'),
 tensor([1.9158], device='cuda:0'),
 tensor([2.4935], device='cuda:0'),
 tensor([1.9972], device='cuda:0'),
 tensor([1.1767], device='cuda:0'),
 tensor([-0.0074], device='cuda:0'),
 tensor([1.5601], device='cuda:0'),
 tensor([1.7741], device='cuda:0'),
 tensor([1.5132], device='cuda:0'),
 tensor([3.1334], device='cuda:0'),
 tensor([0.1628], device='cuda:0'),
 tensor([0.0309], device='cuda:0'),
 tensor([2.2070], device='cuda:0'),
 tensor([0.3566], device='cuda:0'),
 tensor([-0.0318], device='cuda:0'),
 tensor([0.9992], device='cuda:0'),
 tensor([-0.0438], device='cuda:0'),
 tensor([0.0508], device='cuda:0'),
 tensor([0.0817], device='cuda:0'),
 tensor([0.0129], device='cuda:0'),
 tensor([1.7322], device='cuda:0'),
 tensor([0.0191], device='cuda:0'),
 tensor([1.6201], device='cuda:0'),
 tensor([0.9304], device='cuda:0'),
 tensor([0.0678], device=

In [7]:


preds = []
for val in y_pred:
    preds.append(val.detach().tolist()[0])



In [8]:
true = []
for val in y_true:
    true.append(val.detach().tolist()[0])

In [9]:
print(len(true), len(preds))

366 366


In [10]:
predictions = pd.DataFrame(columns = ['y_true', 'y_pred'])
predictions['y_true'] = true
predictions['y_pred'] = preds


In [11]:
predictions

,y_true,y_pred
0,2,2.293459
1,2,2.144251
2,0,0.058913
3,2,1.111601
4,2,1.915846
...,...,...
361,0,0.129442
362,2,1.729738
363,0,0.060487
364,0,0.039030


In [12]:
predictions[predictions['y_true'] == 0].describe().T

,count,mean,std,min,25%,50%,75%,max
y_true,180.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
y_pred,180.0,0.059453,0.130876,-0.072784,-0.004005,0.039171,0.077045,1.142484


In [13]:
predictions[predictions['y_true'] == 1].describe().T

,count,mean,std,min,25%,50%,75%,max
y_true,37.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
y_pred,37.0,1.158972,0.514563,0.064508,0.915378,1.165117,1.519818,2.354398


In [14]:
predictions[predictions['y_true'] == 2].describe().T

,count,mean,std,min,25%,50%,75%,max
y_true,100.0,2.000000,0.000000,2.000000,2.00000,2.000000,2.000000,2.000000
y_pred,100.0,2.134732,0.565186,0.356645,1.84177,2.029556,2.347543,4.158382


In [15]:
predictions[predictions['y_true'] == 3].describe().T

,count,mean,std,min,25%,50%,75%,max
y_true,20.0,3.000000,0.000000,3.000000,3.000000,3.000000,3.000000,3.000000
y_pred,20.0,2.724277,0.460715,1.841188,2.402872,2.838879,3.099932,3.331756


In [16]:
predictions[predictions['y_true'] == 4].describe().T

,count,mean,std,min,25%,50%,75%,max
y_true,29.0,4.000000,0.00000,4.000000,4.000000,4.000000,4.000000,4.000000
y_pred,29.0,2.846631,0.62166,1.762093,2.394118,2.825711,3.409058,4.028105


In [17]:
coef = [0.5, 1.5, 2.5, 3.5]
test_preds = []
for i, pred in enumerate(predictions['y_pred'].values):
    if pred < coef[0]:
        test_preds.append(0)
    elif pred >= coef[0] and pred < coef[1]:
        test_preds.append(1)
    elif pred >= coef[1] and pred < coef[2]:
        test_preds.append(2)
    elif pred >= coef[2] and pred < coef[3]:
        test_preds.append(3)
    else:
        test_preds.append(4)

In [18]:


from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(predictions['y_true'], test_preds, weights = 'quadratic')



0.8977512046265299

: 